# 🧭 GeoDashboard Skeleton v4

Interactieve verkenner voor GPKG-bestanden, met kaart, metadata en layerselectie.

In [1]:
# 📦 Vereiste packages checken
import importlib
packages = ['geopandas', 'fiona', 'shapely', 'matplotlib']
for pkg in packages:
    try:
        importlib.import_module(pkg)
        print(f'✅ {pkg} geïnstalleerd')
    except ImportError:
        print(f'⚠️ {pkg} NIET geïnstalleerd')

✅ geopandas geïnstalleerd
✅ fiona geïnstalleerd
✅ shapely geïnstalleerd
✅ matplotlib geïnstalleerd


In [2]:
# 📂 Paden instellen + modules laden
import os, sys
from pathlib import Path

project_root = Path.cwd()
while not (project_root / 'config.py').exists() and project_root != project_root.parent:
    project_root = project_root.parent
sys.path.insert(0, project_root.as_posix())

from config import SRC_DIR, SHAPEFILES_DIR
if SRC_DIR.as_posix() not in sys.path:
    sys.path.insert(0, SRC_DIR.as_posix())

import geopandas as gpd
import matplotlib.pyplot as plt
from gpkg_service import list_gpkg_files, list_gpkg_layers, load_gpkg_layer

In [3]:
# 🔽 Selectie widgets + interactiviteit
from config import SHAPEFILES_DIR
import ipywidgets as widgets
from IPython.display import display, clear_output

plot_output = widgets.Output()
meta_output = widgets.Output()

gpkg_selector = widgets.Dropdown(
    options=list_gpkg_files(SHAPEFILES_DIR),
    description='GPKG:',
    layout=widgets.Layout(width='400px')
)

layer_selector = widgets.Dropdown(
    options=[],
    description='Layer:',
    layout=widgets.Layout(width='400px')
)

def update_layers(change):
    if gpkg_selector.value:
        layer_selector.options = list_gpkg_layers(gpkg_selector.value)
        if layer_selector.options:
            layer_selector.value = layer_selector.options[0]

def update_outputs(change):
    plot_output.clear_output()
    meta_output.clear_output()
    with plot_output:
        try:
            gdf = load_gpkg_layer(gpkg_selector.value, layer_selector.value)
            gdf.plot(figsize=(8, 8), edgecolor='black')
            plt.title(f"{gpkg_selector.value} – {layer_selector.value}")
            plt.axis('off')
            plt.show()
        except Exception as e:
            print("Fout bij plotten:", e)
    with meta_output:
        try:
            print(f"CRS: {gdf.crs}")
            print(f"Aantal features: {len(gdf)}")
            print(f"Kolommen: {list(gdf.columns)}")
            print(f"Geometry type: {gdf.geom_type.unique()}")
            print(f"Bounding box: {gdf.total_bounds}")
        except Exception as e:
            print("Fout bij metadata:", e)

gpkg_selector.observe(update_layers, names='value')
layer_selector.observe(update_outputs, names='value')
update_layers(None)
update_outputs(None)

display(gpkg_selector, layer_selector)
display(plot_output)
display(meta_output)

Dropdown(description='GPKG:', layout=Layout(width='400px'), options=(PosixPath('/Users/hvingen/Repositories/OU…

Dropdown(description='Layer:', layout=Layout(width='400px'), options=('sample_buurten_10',), value='sample_buu…

Output()

Output()